In [1]:
#import useful libraries 
import pandas as pd
import json
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm 
%matplotlib inline 


##link to the dataset
## https://www.cs.cmu.edu/~ark/personas/

In [150]:
#create path directory
DATA_FOLDER = 'MovieSummaries/'

CHARACTER_DATASET = DATA_FOLDER+"character.metadata.tsv"
MOVIE_DATASET = DATA_FOLDER+"movie.metadata.tsv"
NAME_CLUSTER_DATASET = DATA_FOLDER+"name.clusters.txt"
PLOT_SUMMARY_DATASET = DATA_FOLDER+"plot_summaries.txt"
TROPES_CLUSTER_DATASET = DATA_FOLDER+"tvtropes.clusters.txt"

additional_data_folder = 'archive/'

movie_metadata = additional_data_folder+"movies_metadata.csv"
movie_rating_small = additional_data_folder+"ratings_small.csv"

rating_data_folder = 'ml-latest/'
movie_id = rating_data_folder+"movies.csv"
rating = rating_data_folder+"ratings.csv"

## Load the different datasets

In [64]:
#character dataset
df_character = pd.read_csv(CHARACTER_DATASET, sep='\t', header=None)
column_names_character = ['Wikipedia movie ID', 'Freebase movie ID', 'Release date', 'Character name', 'Birth date', 'Gender', \
                          'height [m]', 'Ethnicity (Freebase ID)', 'Actor name', 'Age at movie release', \
                          'Freebase character/actor map ID', 'Freebase character ID', 'Freebase actor ID']
df_character.columns = column_names_character
display(df_character)

,Wikipedia movie ID,Freebase movie ID,Release date,Character name,Birth date,Gender,height [m],Ethnicity (Freebase ID),Actor name,Age at movie release,Freebase character/actor map ID,Freebase character ID,Freebase actor ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l
3,975900,/m/03vyhn,2001-08-24,Sgt Jericho Butler,1967-09-12,M,1.750,NaN,Jason Statham,33.0,/m/02vchl6,/m/0bgchnq,/m/034hyc
4,975900,/m/03vyhn,2001-08-24,Bashira Kincaid,1977-09-25,F,1.650,NaN,Clea DuVall,23.0,/m/02vbb3r,/m/0bgchp9,/m/01y9xg
...,...,...,...,...,...,...,...,...,...,...,...,...,...
450664,913762,/m/03pcrp,1992-05-21,Elensh,1970-05,F,NaN,NaN,Dorothy Elias-Fahn,NaN,/m/0kr406c,/m/0kr406h,/m/0b_vcv
450665,913762,/m/03pcrp,1992-05-21,Hibiki,1965-04-12,M,NaN,NaN,Jonathan Fahn,27.0,/m/0kr405_,/m/0kr4090,/m/0bx7_j
450666,28308153,/m/0cp05t9,1957,NaN,1941-11-18,M,1.730,/m/02w7gg,David Hemmings,15.0,/m/0g8ngmc,NaN,/m/022g44
450667,28308153,/m/0cp05t9,1957,NaN,NaN,NaN,NaN,NaN,Roberta Paterson,NaN,/m/0g8ngmj,NaN,/m/0g8ngmm


In [123]:
#movie dataset
df_movie = pd.read_csv(MOVIE_DATASET, sep='\t', header=None)
column_names_movie = ['Wikipedia ID', 'Freebase ID', 'Name', 'Release date', 'Box office', 'Runtime', \
                 'Languages (Freebase ID:name tuples)', 'Countries (Freebase ID:name tuples)', \
                 'genres (Freebase ID:name tuples)']
df_movie.columns = column_names_movie
display(df_movie)

,Wikipedia ID,Freebase ID,Name,Release date,Box office,Runtime,Languages (Freebase ID:name tuples),Countries (Freebase ID:name tuples),genres (Freebase ID:name tuples)
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic..."
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}"
...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}"
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0..."
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}"
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ..."


In [130]:
#name cluster dataset
df_name_cluster = pd.read_csv(NAME_CLUSTER_DATASET, delimiter=r'\s+/', header=None, engine='python')
column_name_cluster = ['Actor name', 'Freebase character/actor map ID']

df_name_cluster.columns = column_name_cluster
#display(df_name_cluster)

In [131]:
#read the text file line by line
with open(PLOT_SUMMARY_DATASET, 'r', encoding='utf-8') as file:
    lines = file.readlines()
    
column1 = []
column2 = []

for line in lines:
    #split on the first space
    parts = line.split(' ', 1)  
    if len(parts) == 2:
        part1 = parts[0].strip()
        part2 = parts[1].strip()
        #check if the first part is a number
        if part1.isdigit():  
            column1.append(part1)
            column2.append(part2)

df_summary = pd.DataFrame({'Wikipedia ID movie': column1, 'Summary': column2})

#display(df_summary)

In [132]:
#read again the text file line by line
with open(TROPES_CLUSTER_DATASET, 'r') as file:
    lines = file.readlines()
    
#create empty lists to store extracted data
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

for line in lines:
    parts = line.strip().split('\t')
    if len(parts) == 2:
        #extract the first part
        column1.append(parts[0])
        #parse the JSON string in the second part
        data = json.loads(parts[1])
        # Extract the different values 
        column2.append(data.get('char', ''))  
        column3.append(data.get('movie', ''))  
        column4.append(data.get('id', '')) 
        column5.append(data.get('actor', '')) 
        
df_tropes_cluster = pd.DataFrame({'Character types': column1, 'Char': column2, 'Movie': column3, 'Freebase character/actor map ID': column4, 'Actor': column5})

#display(df_tropes_cluster)

## Process data and first analysis

In [133]:
#convert the 'Release date' column of movie dataset to YYYY-MM-DD format with 3 new columns : 'Year', 'Day'

#initialize new columns for Year, Month, and Day
df_movie['Year'] = np.nan
df_movie['Month'] = np.nan
df_movie['Day'] = np.nan

#extract Year, Month, and Day based on the string format
for index, release_date in enumerate(df_movie['Release date']):
    #if date_str is NaN, Year, Month, and Day remain NaN
    if pd.notnull(release_date):
        date_parts = str(release_date).split('-')
        df_movie.at[index, 'Year'] = int(date_parts[0])
        #if only the year is available, Month and Day are left as NaN
        if len(date_parts) == 3:  # Full date is present
            df_movie.at[index, 'Month'] = int(date_parts[1])
            df_movie.at[index, 'Day'] = int(date_parts[2])
            
#convert columns to nullable integer types
df_movie['Year'] = df_movie['Year'].astype('Int64')
df_movie['Month'] = df_movie['Month'].astype('Int64')
df_movie['Day'] = df_movie['Day'].astype('Int64')


#display(df_movie)

In [134]:
percentage_missing_values_year = (df_movie['Year'].isna().sum()/len(df_movie['Year']))*100
print(f"The percentage of missing values year release date is {format(percentage_missing_values_year, '.3f')}%.")

percentage_missing_values_month = (df_movie['Month'].isna().sum()/len(df_movie['Month']))*100
print(f"The percentage of missing values month release date (and therefore day) is {format(percentage_missing_values_month, '.3f')}%.")

percentage_missing_values_revenues = (df_movie['Box office'].isna().sum()/len(df_movie['Box office']))*100
print(f"The percentage of missing values Box office is {format(percentage_missing_values_revenues, '.3f')}%.")

The percentage of missing values year release date is 8.444%.
The percentage of missing values month release date (and therefore day) is 51.832%.
The percentage of missing values Box office is 89.722%.


## Add addtionnal datasets and merge what we need 

In [139]:
#datasets from archive folder
df_movie_metadata = pd.read_csv(movie_metadata, sep=',', header=0)
df_movie_metadata.rename(columns={'title': 'Name', 'revenue': 'Box office'}, inplace=True)
df_movie_metadata['Box office'] = df['Box office'].replace(0, np.nan)

#display(df_movie_metadata)

C:\Users\albia\AppData\Local\Temp\ipykernel_6244\1725085157.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movie_metadata = pd.read_csv(movie_metadata, sep=',', header=0)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,Box office,runtime,spoken_languages,status,tagline,Name,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,NaN,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...",http://www.imdb.com/title/tt6209470/,439050,tt6209470,fa,رگ خواب,Rising and falling between a man and woman.,...,NaN,NaN,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,False,NaN,0,"[{'id': 18, 'name': 'Drama'}]",NaN,111109,tt2028550,tl,Siglo ng Pagluluwal,An artist struggles to finish his work while a...,...,2011-11-17,NaN,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,67758,tt0303758,en,Betrayal,"When one of her hits goes wrong, a professiona...",...,2003-08-01,NaN,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,False,NaN,0,[],NaN,227506,tt0008536,en,Satana likuyushchiy,"In a small town live two brothers, one a minis...",...,1917-10-21,NaN,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [148]:
#datasets from ml-latest folder
df_movie_id = pd.read_csv(movie_id, sep=',', header=0)
df_rating = pd.read_csv(rating, sep=',', header=0)
#display(df_movie_id)
#display(df_rating)

In [149]:
#remove release year from title
df_movie_id['title'] = df_movie_id['title'].str.replace(r'\s\(\d{4}\)', '', regex=True)
#display(df_movie_id)

#merged the datasets in movieId
merged_df = df_movie_id.merge(df_rating, on='movieId', how='inner')

#as multiple rating for one movie, group by title
df_grouped = merged_df.groupby('title').agg(mean_rating=('rating', 'mean')).reset_index()
display(df_grouped)

,title,mean_rating
0,,2.675676
1,"""BLOW THE NIGHT!"" Let's Spend the Night Together",2.000000
2,"""Great Performances"" Cats",2.912727
3,"""Sr.""",3.227273
4,#1 Cheerleader Camp,2.166667
...,...,...
77208,牛づれ超特急,2.000000
77209,茶叶之旅,3.000000
77210,貞子3D,2.250000
77211,过昭关,3.500000


In [ ]:
##merged_df_1 = df_movie.merge(df_grouped, on='Name', how='left')

## Research on release month and revenues 

In [37]:
#first drop nan month release date values 
df_movie.dropna(subset=['Box office', 'Month'],inplace=True)
display(df_movie)

,Wikipedia ID,Freebase ID,Name,Release date,Box office,Runtime,Languages (Freebase ID:name tuples),Countries (Freebase ID:name tuples),genres (Freebase ID:name tuples),Year,Month,Day
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001,8,24
7,10408933,/m/02qc0j7,Alexander's Ragtime Band,1938-08-16,3600000.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/04t36"": ""Musical"", ""/m/01z4y"": ""Comedy"", ...",1938,8,16
13,171005,/m/016ywb,Henry V,1989-11-08,10161099.0,137.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/04xvh5"": ""Costume drama"", ""/m/082gq"": ""Wa...",1989,11,8
17,77856,/m/0kcn7,Mary Poppins,1964-08-27,102272727.0,139.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3myq"": ""Children's/Family"", ""/m/04t36""...",1964,8,27
21,612710,/m/02wjqm,New Rose Hotel,1999-10-01,21521.0,92.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",1999,10,1
...,...,...,...,...,...,...,...,...,...,...,...,...
81677,6740261,/m/0glmbl,Eternally Yours,1939-10-07,683131.0,91.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02h8pkk"": ""Family Drama"", ""/m/01g6gs"": ""B...",1939,10,7
81695,54540,/m/0f7hw,Coming to America,1988-06-29,288752301.0,117.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/03p5xs"": ""...",1988,6,29
81725,1918494,/m/0660qx,State and Main,2000-08-26,6944471.0,106.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/0f8l9c"": ""France"", ""/m/09c7w0"": ""United S...","{""/m/0gf28"": ""Parody"", ""/m/09n5t_"": ""Americana...",2000,8,26
81726,664006,/m/030xw6,Guilty as Sin,1993-06-04,22886222.0,107.0,{},"{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1993,6,4


In [57]:
movie_revenues = df_movie['Box office'].values
month_release = pd.get_dummies(df_movie['Month']).values
#remove one month column to select it as a baseline here january
month_release = month_release[:, :-1].astype(int)
month_release = sm.add_constant(month_release)

In [58]:
model = sm.OLS(movie_revenues, month_release).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.044
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     28.53
Date:                Fri, 03 Nov 2023   Prob (F-statistic):           2.89e-59
Time:                        15:50:25   Log-Likelihood:            -1.3600e+05
No. Observations:                6794   AIC:                         2.720e+05
Df Residuals:                    6782   BIC:                         2.721e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        7.26e+07   4.57e+06     15.884      0.000    6.36e+07    8.16e+07
x1         -4.706e+07   6.96e+06     -6.760      0.000   -6.07e+07   -3.34e+07
x2         -3.213e+07   7.12e+06     -4.516      0.000   -4.61e+07   -1.82e+07
x3         -2.678e+07   6.84e+06     -3.914      0.000   -4.02e+07   -1.34e+07
x4          -2.75e+07   6.94e+06     -3.963      0.000   -4.11e+07   -1.39e+07
x5          1.225e+07   6.86e+06      1.787      0.074   -1.19e+06    2.57e+07
x6          4.107e+07   7.08e+06      5.799      0.000    2.72e+07     5.5e+07
x7          1.313e+07   7.05e+06      1.863      0.063   -6.87e+05    2.69e+07
x8         -3.579e+07   6.69e+06     -5.346      0.000   -4.89e+07   -2.27e+07
x9          -4.12e+07   6.29e+06     -6.554      0.000   -5.35e+07   -2.89e+07
x10        -3.314e+07    6.6e+06     -5.020      0.000   -4.61e+07   -2.02e+07
x11          3.87e+06   7.03e+06      0.550      0.582   -9.92e+06    1.77e+07
==============================================================================
Omnibus:                     8057.610   Durbin-Watson:                   2.014
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          1575108.603
Skew:                           6.091   Prob(JB):                         0.00
Kurtosis:                      76.591   Cond. No.                         11.9
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [121]:
# Create a mapping of movie names to their box office values from df2
box_office_map = df.set_index('Name')['Box office'].to_dict()

# Fill NaN values in 'box office' column of df1 using the map
mask = df_movie['Box office'].isna()  # Identify NaN values in 'box office' column of df1
df_movie.loc[mask, 'Box office'] = df_movie.loc[mask, 'Name'].map(box_office_map)

display(df_movie)

71301


,Wikipedia ID,Freebase ID,Name,Release date,Box office,Runtime,Languages (Freebase ID:name tuples),Countries (Freebase ID:name tuples),genres (Freebase ID:name tuples),Year,Month,Day
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science...",2001,8,24
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp...",2000,2,16
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D...",1988,<NA>,<NA>
3,9363483,/m/0285_cd,White Of The Eye,1987,NaN,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/07ssc"": ""United Kingdom""}","{""/m/01jfsb"": ""Thriller"", ""/m/0glj9q"": ""Erotic...",1987,<NA>,<NA>
4,261236,/m/01mrr1,A Woman in Flames,1983,NaN,106.0,"{""/m/04306rv"": ""German Language""}","{""/m/0345h"": ""Germany""}","{""/m/07s9rl0"": ""Drama""}",1983,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...
81736,35228177,/m/0j7hxnt,Mermaids: The Body Found,2011-03-19,NaN,120.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama""}",2011,3,19
81737,34980460,/m/0g4pl34,Knuckle,2011-01-21,NaN,96.0,"{""/m/02h40lc"": ""English Language""}","{""/m/03rt9"": ""Ireland"", ""/m/07ssc"": ""United Ki...","{""/m/03bxz7"": ""Biographical film"", ""/m/07s9rl0...",2011,1,21
81738,9971909,/m/02pygw1,Another Nice Mess,1972-09-22,NaN,66.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06nbt"": ""Satire"", ""/m/01z4y"": ""Comedy""}",1972,9,22
81739,913762,/m/03pcrp,The Super Dimension Fortress Macross II: Lover...,1992-05-21,NaN,150.0,"{""/m/03_9r"": ""Japanese Language""}","{""/m/03_3d"": ""Japan""}","{""/m/06n90"": ""Science Fiction"", ""/m/0gw5n2f"": ...",1992,5,21
